## Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [2]:
boston = load_boston()
df =pd.DataFrame(boston.data, columns=boston.feature_names)
df['target']=boston.target

In [3]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [4]:
df.shape

(506, 14)

In [5]:
df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
target     0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


### 1. Implementaton of Regression Model

In [7]:
X = df.drop('target',axis=1)
y = df['target']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.20, random_state=33)
dtr =DecisionTreeRegressor()

dtr.fit(x_train, y_train)

DecisionTreeRegressor()

In [9]:
print(dtr.score(x_test, y_test))

0.686890790947211


In [10]:
pram_grid = {'bootstrap':[True],'max_depth':[5,10,None],'max_features':['auto','log2'],
             'n_estimators':[5,6,7,8,9,10,11,12,13,14,15]}

### 2. Implementation of Model using GridSearchCV

In [11]:
rfr = RandomForestRegressor(random_state=1)

g_search = GridSearchCV(estimator=rfr, param_grid=pram_grid, cv=3, n_jobs=1, verbose=0, return_train_score=True)

In [12]:
g_search.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=1), n_jobs=1,
             param_grid={'bootstrap': [True], 'max_depth': [5, 10, None],
                         'max_features': ['auto', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                          15]},
             return_train_score=True)

In [13]:
print(g_search.best_params_)

{'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 9}


In [14]:
best_grid = g_search.best_estimator_
print(best_grid.score(x_test,y_test))

0.8156958718509653


### 3. Implementation of model using RandomSearchCV

In [15]:
r_grid = {'n_estimators': list(range(5,15)),'max_features' : ['auto','log2'],
          'max_depth' : [5,10, None],'bootstrap' : [True, False]}

rfr_random = RandomizedSearchCV(estimator=rfr, param_distributions=r_grid, n_iter=20,scoring='neg_mean_absolute_error',
                               cv=3, verbose=2, random_state=34, n_jobs=-1, return_train_score=True)

rfr_random.fit(x_train,y_train)

print(rfr_random.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
{'n_estimators': 13, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': True}


In [16]:
best_random = rfr_random.best_estimator_
print(best_random.score(x_test,y_test))

0.8092784575584067


In [17]:
#end